In [61]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import re
from multiprocessing import Pool, Lock, Value
from time import sleep
import os

In [6]:
queries = pd.read_csv('text-relevance-competition-ir-1-ts-spring-2020/queries.numerate.txt', sep='	', header=None)
urls = pd.read_csv('text-relevance-competition-ir-1-ts-spring-2020/urls.numerate.txt', sep='	', header=None)
urls.index = urls[1]

In [116]:
doc0 = open('text-relevance-competition-ir-1-ts-spring-2020/content/20170702/doc.0000.dat')

html = ' '.join(list(doc0))

'povarito.ru/1403-manty-s-tykvoiy-poshagovyiy-foto-recept.html'

In [80]:
docnames = []

for d in listdir:
    listdocs = os.listdir('text-relevance-competition-ir-1-ts-spring-2020/content/' + d)
    try:
        listdocs.remove('.DS_Store')
    except ValueError:
        pass
    listdocs.sort()
    docnames += list(map(lambda x: d + '/' + x, listdocs))
docnames[-5:]

['20170726/doc.5045.dat',
 '20170726/doc.5046.dat',
 '20170726/doc.5047.dat',
 '20170726/doc.5048.dat',
 '20170726/doc.5049.dat']

In [117]:
def doc2words(docname):
    with open('text-relevance-competition-ir-1-ts-spring-2020/content/' + docname, errors='ignore') as read_file:
        html = ' '.join(list(read_file))
    soup = BeautifulSoup(html)
    url = re.sub("http://", "", soup.body.p.text.split()[0])
    doc_id = urls.at[url, 0]
    if soup.text:
        text = re.sub(r'[^A-Za-zА-Яа-я\.\-]+', ' ', soup.text).split()
    else:
        text = []
    if soup.title:
        if soup.title.text:
            title = re.sub(r'[^A-Za-zА-Яа-я\.\-]+', ' ', soup.title.text).split()
        else:
            title = []
    else:
        title = []

    return (title + text, doc_id)

In [ ]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(docname):
    res = doc2words(docname) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    docs = pool.map(func_wrapper, docnames)

9170 objects are processed...

In [ ]:
doc_text = list(map(lambda x: x[0], docs))
doc_id = list(map(lambda x: x[1], docs))

doc_df = {id: docs_id, text: docs}
doc_df.to_csv("text_relevance.csv")

### tf-idf